In [1]:
## Autoreloading libraries https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

from IPython.core.display import display
from IPython.core.display import HTML
import pandas as pd


from talon.signature import EXTRACTOR_FILENAME, EXTRACTOR_DATA
from talon.signature.learning.classifier import train, init
import talon.signature.learning.helpers as helpers
import talon.quotations as tq

from talon.quotations import register_xpath_extensions

import talon.utils as utils

register_xpath_extensions()
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_parquet('./data/part-00001.snappy.parquet')
data = data[data['status'] == '2']
data = data.reset_index()
#data = data[4:5]
data.head()

,index,id,parent_id,activity_id,created_by_id,created_date,last_modified_date,last_modified_by_id,system_modstamp,text_body,html_body,headers,subject,from_name,from_address,validated_from_address,to_address,cc_address,bcc_address,incoming,has_attachment,status,message_date,is_deleted,reply_to_email_message_id,is_externally_visible
0,1,02s2o00003EtrRrAAJ,5002o00002Wqif5AAB,None,005D00000041R4YIAU,2020-09-07T00:45:08.000+0000,2020-09-07 00:45:08,005D00000041R4YIAU,2020-09-07 02:36:14,Dear\n204651349 箱量请改为2X40HQ，谢谢！\n\n\n\nBest R...,"<html><head><meta http-equiv=""content-type"" co...",None,204651349,grace.shao,grace.shao@uniwill56.com,grace.shao@uniwill56.com,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 00:43:38,None,None,None
1,3,02s2o00003Eub4MAAR,5002o00002Wqs59AAB,None,005D00000041R4YIAU,2020-09-07T06:00:01.000+0000,2020-09-07 06:00:01,005D00000041R4YIAU,2020-09-07 06:37:25,亿通显示订舱号错误204647537\n\n\n\nHarry_HaoYue@163.com,"<html><head><meta http-equiv=""content-type"" co...",None,亿通显示订舱号错误204647537,Harry_HaoYue@163.com,harry_haoyue@163.com,harry_haoyue@163.com,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 05:59:49,None,None,None
2,5,02s2o00003EujnIAAR,5002o00002WqwrPAAR,None,005D00000041R4YIAU,2020-09-07T07:22:52.000+0000,2020-09-07 07:22:52,005D00000041R4YIAU,2020-09-07 07:39:30,DEAR ALL\n\n\n204632591\n204632387\n\n\n\n\n\n...,"<html>\n<head>\n <meta http-equiv=""Content-...",None,ningbo-lome 签单付款,ZHENGTIAN-LILY,zhejiangzhengtian@163.com,zhejiangzhengtian@163.com,twillsupport@maersk.com,stella.ge@twill.net; 2126347372@qq.com,None,True,None,2,2020-09-07 07:22:40,None,None,None
3,7,02s2o00003EurtyAAB,5002o00002WqSt1AAF,None,005D00000041R4YIAU,2020-09-07T08:38:40.000+0000,2020-09-07 09:55:00,00557000007smniAAA,2020-09-07 09:56:05,"Dear Claudia,\n\nOK.\nSince my & my supplier (...","<html>\n <head>\n <meta http-equiv=""Conten...",In-Reply-To: <Ih5mR000000000000000000000000000...,"Re: No able to book, error [ ref:_00DD0n6xF._5...",Drago Lackovic,drago@lackovic.hr,drago@lackovic.hr,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 08:38:29,None,None,None
4,11,02s2o00003EvCHIAA3,5002o00002WCTHeAAP,None,005D00000041R4YIAU,2020-09-07T12:40:39.000+0000,2020-09-07 20:14:01,0052o000009bcD0AAI,2020-09-07 20:14:43,Hi Maersk Twill Team\n\nPLease reply as soon a...,"<div dir=""ltr"">Hi Maersk Twill Team<div><br></...",In-Reply-To: <5hYXu000000000000000000000000000...,Re: 204601228 - JEBEL ALI TO EGYPT [ ref:_00DD...,Mansoor Byari,grm@nejoumaljazeera.com,grm@nejoumaljazeera.com,twillsupport@maersk.com,vikramjeet.karangale@maersk.com; booking@nejou...,None,True,None,2,2020-09-07 12:40:23,None,None,None


## Looking into the data

Looking into the dataset and understanding the 

In [84]:
data['html_body_quote_removed'] = data.apply(lambda row: tq.extract_from_html(row['html_body']) if row['html_body'] else None, axis=1)
data['text_body_quote_removed'] = data.apply(lambda row:  tq.extract_from_plain(row['text_body']) if row['text_body'] else None, axis=1)


data.head()

,index,id,parent_id,activity_id,created_by_id,created_date,last_modified_date,last_modified_by_id,system_modstamp,text_body,html_body,headers,subject,from_name,from_address,validated_from_address,to_address,cc_address,bcc_address,incoming,has_attachment,status,message_date,is_deleted,reply_to_email_message_id,is_externally_visible,html_body_quote_removed,text_body_quote_removed
0,1,02s2o00003EtrRrAAJ,5002o00002Wqif5AAB,None,005D00000041R4YIAU,2020-09-07T00:45:08.000+0000,2020-09-07 00:45:08,005D00000041R4YIAU,2020-09-07 02:36:14,Dear\n204651349 箱量请改为2X40HQ，谢谢！\n\n\n\nBest R...,"<html><head><meta http-equiv=""content-type"" co...",None,204651349,grace.shao,grace.shao@uniwill56.com,grace.shao@uniwill56.com,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 00:43:38,None,None,None,"<html><head><meta http-equiv=""content-type"" co...",Dear\n204651349 箱量请改为2X40HQ，谢谢！\n\n\n\nBest R...
1,3,02s2o00003Eub4MAAR,5002o00002Wqs59AAB,None,005D00000041R4YIAU,2020-09-07T06:00:01.000+0000,2020-09-07 06:00:01,005D00000041R4YIAU,2020-09-07 06:37:25,亿通显示订舱号错误204647537\n\n\n\nHarry_HaoYue@163.com,"<html><head><meta http-equiv=""content-type"" co...",None,亿通显示订舱号错误204647537,Harry_HaoYue@163.com,harry_haoyue@163.com,harry_haoyue@163.com,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 05:59:49,None,None,None,"<html><head><meta http-equiv=""content-type"" co...",亿通显示订舱号错误204647537\n\n\n\nHarry_HaoYue@163.com
2,5,02s2o00003EujnIAAR,5002o00002WqwrPAAR,None,005D00000041R4YIAU,2020-09-07T07:22:52.000+0000,2020-09-07 07:22:52,005D00000041R4YIAU,2020-09-07 07:39:30,DEAR ALL\n\n\n204632591\n204632387\n\n\n\n\n\n...,"<html>\n<head>\n <meta http-equiv=""Content-...",None,ningbo-lome 签单付款,ZHENGTIAN-LILY,zhejiangzhengtian@163.com,zhejiangzhengtian@163.com,twillsupport@maersk.com,stella.ge@twill.net; 2126347372@qq.com,None,True,None,2,2020-09-07 07:22:40,None,None,None,"<html>\n<head>\n <meta http-equiv=""Content-...",DEAR ALL\n\n\n204632591\n204632387\n\n\n\n\n\n...
3,7,02s2o00003EurtyAAB,5002o00002WqSt1AAF,None,005D00000041R4YIAU,2020-09-07T08:38:40.000+0000,2020-09-07 09:55:00,00557000007smniAAA,2020-09-07 09:56:05,"Dear Claudia,\n\nOK.\nSince my & my supplier (...","<html>\n <head>\n <meta http-equiv=""Conten...",In-Reply-To: <Ih5mR000000000000000000000000000...,"Re: No able to book, error [ ref:_00DD0n6xF._5...",Drago Lackovic,drago@lackovic.hr,drago@lackovic.hr,twillsupport@maersk.com,None,None,True,None,2,2020-09-07 08:38:29,None,None,None,<html><head>\n \n </head>\n <body>\n <...,"Dear Claudia,\n\nOK.\nSince my & my supplier (..."
4,11,02s2o00003EvCHIAA3,5002o00002WCTHeAAP,None,005D00000041R4YIAU,2020-09-07T12:40:39.000+0000,2020-09-07 20:14:01,0052o000009bcD0AAI,2020-09-07 20:14:43,Hi Maersk Twill Team\n\nPLease reply as soon a...,"<div dir=""ltr"">Hi Maersk Twill Team<div><br></...",In-Reply-To: <5hYXu000000000000000000000000000...,Re: 204601228 - JEBEL ALI TO EGYPT [ ref:_00DD...,Mansoor Byari,grm@nejoumaljazeera.com,grm@nejoumaljazeera.com,twillsupport@maersk.com,vikramjeet.karangale@maersk.com; booking@nejou...,None,True,None,2,2020-09-07 12:40:23,None,None,None,"<div dir=""ltr"">Hi Maersk Twill Team<div><br></...",Hi Maersk Twill Team\n\nPLease reply as soon a...


In [85]:
# TODO FIX ME -> html_body to text_body seems not to work properly.......
# ANSWER -> html_body parser only accepts _MAX_TAGS_COUNT size in HTML tags therefore too big email chains are not processed. (see def: html_document_fromstring)
data['has_signature'] = data.apply(lambda row: helpers.has_signature(row['text_body_quote_removed'], row['validated_from_address']) if row['text_body_quote_removed'] else 'Empty', axis=1)


In [86]:
data['has_signature'].value_counts(dropna=False)

True     1900
Empty     788
NaN       775
Name: has_signature, dtype: int64

In [66]:
no_signature = data[data['has_signature'].isnull()].reset_index()

## Evaluating the `has_signature` function

In [10]:
error_index = 19
print('Email: ', no_signature['validated_from_address'][error_index])
print(no_signature['text_body_quote_removed'][error_index])
print()
print('HTML EMAIL ------------')
print()
display(HTML(no_signature['html_body_quote_removed'][error_index]))


Email:  snapfreight@gmail.com
Dear Mr.Deepak,

Please Note as per CHA form 13 is issued and the container is Gated in.
But on Maersk's website there is no update.
Can you please check and confirm the container gated in or not?

HTML EMAIL ------------



In [55]:
SIGNATURE_MAX_LINES = 16

def has_signature2(body, sender):
    '''Checks if the body has signature. Returns True or False.'''
    # split signature long lines e.g. Tel: +34 123 456 789 | Fax: +34 123 456 789
    non_empty = helpers.flatten_list([line.split(' | ') for line in body.splitlines() if line.strip()])
    """
    Removing long lines from emails before looping through candidates as sometimes we have:
    >>> Jean Phelippe
    >>> ABC Logistics
    >>> Germany
    >>> Wir arbeiten ausschließlich auf Grundlage der Allgemeinen Deutschen Spediteurbedingungen, ADSp2017.
    >>> This e-mail and any attachments are confidential and may also be privileged. If you are not the named recipient, please notify the sender immediately and do not disclose the contents to another person[...]
    """
    not_long = [line for line in non_empty if len(line) <= 200]
    candidates = not_long[-SIGNATURE_MAX_LINES:]
    print('Candidates: ')
    print(candidates)
    print()
    print()
    upvotes = 0
    for line in candidates:
        if helpers.contains_sender_names(sender)(line):
            print('Contain sender: ', line)
            return True
        elif (helpers.binary_regex_search(helpers.RE_RELAX_PHONE)(line) +
              helpers.binary_regex_search(helpers.RE_EMAIL)(line) +
              helpers.binary_regex_search(helpers.RE_URL)(line) >= 1):
              print('Contain RELAX_PHONE, EMAIL OR URL: ', line)
              upvotes += 1
        elif (helpers.binary_regex_search(helpers.RE_SIGNATURE_WORDS)(line) == 1):
            print('Contain RE_SIGNATURE_WORDS: ', line)
            upvotes += 1
        else:
            print('Nothing found ', line)
    if upvotes > 1:
        return True

In [56]:
error_index = 22

print('Email: ', no_signature['validated_from_address'][error_index])
print(no_signature['text_body_quote_removed'][error_index])
print('----------ANALYSIS----------')
has_signature2(no_signature['text_body_quote_removed'][error_index], no_signature['validated_from_address'][error_index])

Email:  shipping1@younsaint.com
Hi,
此单的数据已经提交了，但是柜型之前后台修改为40HC，系统还未同步，立方数我这边是69，系统只能填67先提交了，请后台帮忙更新，谢谢。


Mandy Yan
 
Cell#: 86-18321292926    Tel: 86-21-34635185-802   E-mail: shipping1@younsaint.com
Office: 301, Buliding #40, No 258, Xinzhuan Rd, Shanghai 201612
Showroom: 101, Builiding #42,No 258, Xinzhuan Rd, Shanghai 201612
----------ANALYSIS----------
Candidates: 
['Hi,', '此单的数据已经提交了，但是柜型之前后台修改为40HC，系统还未同步，立方数我这边是69，系统只能填67先提交了，请后台帮忙更新，谢谢。', 'Mandy Yan', 'Cell#: 86-18321292926    Tel: 86-21-34635185-802   E-mail: shipping1@younsaint.com', 'Office: 301, Buliding #40, No 258, Xinzhuan Rd, Shanghai 201612', 'Showroom: 101, Builiding #42,No 258, Xinzhuan Rd, Shanghai 201612']


Nothing found  Hi,
Nothing found  此单的数据已经提交了，但是柜型之前后台修改为40HC，系统还未同步，立方数我这边是69，系统只能填67先提交了，请后台帮忙更新，谢谢。
Nothing found  Mandy Yan
Contain RELAX_PHONE, EMAIL OR URL:  Cell#: 86-18321292926    Tel: 86-21-34635185-802   E-mail: shipping1@younsaint.com
Contain RELAX_PHONE, EMAIL OR URL:  Office: 301, Buliding #40, No

True

In [75]:
helpers.binary_regex_search(helpers.RE_EMAIL)('Cell#: 86-18321292926    Tel: 86-21-34635185-802   E-mail: shipping1@younsaint.com')

1

In [30]:
#display(HTML(no_signature['html_body'][2]))
error_index = 2
#helpers.has_signature(no_signature['validated_text_body'][error_index], no_signature['validated_from_address'][error_index])
print(no_signature['validated_from_address'][error_index])
print()
[line for line in no_signature['validated_text_body'][error_index].splitlines() if line.strip()]

phuongtth@phaleminerals.com.vn



['Dear Ms Nhã,',
 'B/L thêm giúp em vào phần mổ tả hàng hóa:',
 '-          NOT FOR MEDICINAL USE ',
 '-          HS CODE: 38249990',
 'Các nội dung còn lại ok rồi ạ.',
 '         ********',
 'Thank and Best regard!                                              ',
 'HA PHUONG TRAN',
 'Imp – Exp Department',
 'Mobile   : +84 936 093 337',
 'Skype    : haphuong.mars3',
 'Website :   <http://www.phaleminerals.com.vn/> www.phaleminerals.com.vn',
 'Address :  2th Floor, Tower A, Rivera Park , 69  Vu Trong Phung Str.,  Thanh Xuan Dist., Hanoi, Vietnam']

In [67]:
import email

email.message_from_string(html_example)

'Hi Ms Nha,\n\n \n\nSorry that customer have cancelled below booking , thank you.\n\n \n\nBest Regards\n\nJudy'

In [76]:
signature_data = data[data['has_signature'] == True]

In [77]:
signature_data['text_body_quote_removed'][0].splitlines()

['Dear',
 '204651349  箱量请改为2X40HQ，谢谢！',
 '',
 '',
 '',
 'Best Regards ',
 '-------------- ',
 '邵    娜   Grace',
 '常州分公司     海运部  ',
 '江苏众诚国际物流有限公司',
 'Jiangsu Uniwill Logistics Co.,Ltd.',
 'Tel: 86-519-85167019',
 'Fax: 86-519-85167011',
 'Mobile: 13961144307',
 '企业微信: 联系我',
 'QQ:310634359',
 'Add: 江苏省常州市新北区府西花园商用房6-301室',
 'ZIP: 213022',
 'ADD: 6-301 COMMERCIAL BUILDING OF FUXI GARDEN,CHANGZHOU NEW DISTRICT CHANGZHOU,CHINA',
 'Website: http://www.uniwill-logistics.com']

In [89]:
SIGNATURE_MAX_LINES = 16

def tag_signature(body, sender, TAG='#sig#'):
    '''Tags the signature'''
    """
        TODO -> 1) index all lines of the body, 2) run through the program and get first potential candidate, tag all rows till the end as signature.
    """
    all_lines = body.splitlines()
    index = None
    # Get first signature line
    for line in all_lines:
        if helpers.contains_sender_names(sender)(line):
            print('contains sender')
            index = all_lines.index(line)
            break
        elif (helpers.binary_regex_search(helpers.RE_RELAX_PHONE)(line) +
              helpers.binary_regex_search(helpers.RE_EMAIL)(line) +
              helpers.binary_regex_search(helpers.RE_URL)(line) >= 1):
              print('contains email, url or phone')
              print(line)
              index = all_lines.index(line)
              break
        elif (helpers.binary_regex_search(helpers.RE_SIGNATURE_WORDS)(line) == 1):
            print('contains signature word')
            print(line)
            index = all_lines.index(line)
            break
        else:
            pass
    return index

In [90]:
tag_signature(signature_data['text_body_quote_removed'][0], signature_data['validated_from_address'][0])

contains signature word
Best Regards 


5